In [6]:
!wget https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv -O /root/cvd_open_images/class-descriptions.csv --no-check-certificate


--2018-07-23 17:17:06--  https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11255 (11K) [text/csv]
Saving to: ‘/root/cvd_open_images/class-descriptions.csv’

/root/cvd_open_imag 100%[===================>]  10.99K  --.-KB/s    in 0s      

2018-07-23 17:17:07 (93.0 MB/s) - ‘/root/cvd_open_images/class-descriptions.csv’ saved [11255/11255]



In [9]:
import csv
import os.path as osp
label_id_to_name = {}
DATA = '/root/cvd_open_images'

with open(osp.join(DATA,'class-descriptions.csv')) as f:
    reader = csv.reader(f)
    for line in reader:
        label_id, label_name = line
        label_id_to_name[label_id] = label_name

In [30]:
bool(int('0'))

False

In [115]:
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt

from PIL import Image
import numpy as np

# split = 'validation'
split = 'train'


html = lambda s: display(HTML(s))
def show_by_image_id(image_id):
    img = Image.open(osp.join(DATA, f'{split}/{image_id}.jpg'))
    plt.axis('off')
    plt.imshow(img)
    plt.show()

label_name_to_image_ids = defaultdict(lambda: set())

image_ids = set()
total = 0

labels_file = osp.join(DATA, f"{split}-annotations-human-imagelabels-boxable.csv")
bbox_file = osp.join(DATA, f'{split}-annotations-bbox.csv')

def is_valid(bbox_line):
    image_id, source, label_name, confidence, x_min, x_max, y_min, y_max, is_occluded, is_truncated, is_group_of, is_depiction, is_inside = bbox_line
    
    if int(is_occluded) or int(is_depiction) or int(is_inside) or int(is_group_of):
        return False
    
    if int(is_truncated):
        return False
    
    # Check that the image is not very small
    x_min, x_max, y_min, y_max = float(x_min), float(x_max), float(y_min), float(y_max)
    width = x_max - x_min
    height = y_max - y_min
    
    # Require that the object takes up at least 1/3rd by 1/3rd of the image
    minimum_area = 0.35 * 0.35 #0.12
    area = width * height 
    return area > minimum_area
    
with open(labels_file) as f:
    bbox_reader = csv.reader(open(bbox_file))
    bbox_header = next(bbox_reader)
    
    print(bbox_header)
    for bbox_line in tqdm(bbox_reader):    
        image_id, source, label_id, confidence, x_min, x_max, y_min, y_max, is_occluded, is_truncated, is_group_of, is_depiction, is_inside = bbox_line
        label_name = label_id_to_name[label_id]
        
        if not is_valid(bbox_line):
            continue
            
        label_name_to_image_ids[label_name].add(image_id)
        image_ids.add(image_id)
        
#         print(line)
#         print(bbox_line)
#         print(image_id, source, label_id, confidence)
        
#         if label_name == 'Bicycle':
#             print(label_name)
#             show_by_image_id(image_id)
#             total += 1
            
#         if total > 10:
#             break


label_name_to_counts = {k:len(image_ids) for k, image_ids in label_name_to_image_ids.items()}
print("Bicycles: ", label_name_to_counts['Bicycle'])            
print("Birds: ", label_name_to_counts['Bird'])               
print("Boats: ", label_name_to_counts['Boat'])            


['ImageID', 'Source', 'LabelName', 'Confidence', 'XMin', 'XMax', 'YMin', 'YMax', 'IsOccluded', 'IsTruncated', 'IsGroupOf', 'IsDepiction', 'IsInside']


Bicycles:  1351
Birds:  3476
Boats:  5473


In [119]:
print('n_images %s' % len(image_ids))
print('n_labels %s' % total)

sorted(label_name_to_counts.items(), key=lambda x: -x[1])

n_images 448365
n_labels 0


[('Clothing', 67039),
 ('Person', 30151),
 ('House', 27960),
 ('Car', 26038),
 ('Human body', 15941),
 ('Human face', 15882),
 ('Tree', 14529),
 ('Table', 11805),
 ('Boy', 11287),
 ('Poster', 10873),
 ('Man', 10276),
 ('Tower', 9881),
 ('Flower', 8307),
 ('Woman', 6924),
 ('Suit', 6788),
 ('Girl', 6050),
 ('Boat', 5399),
 ('Sculpture', 5312),
 ('Desk', 4575),
 ('Building', 4095),
 ('Toy', 4082),
 ('Airplane', 3928),
 ('Motorcycle', 3803),
 ('Dessert', 3800),
 ('Food', 3650),
 ('Bird', 3412),
 ('Human head', 3368),
 ('Window', 3272),
 ('Laptop', 3232),
 ('Bus', 3208),
 ('Snack', 3198),
 ('Sports uniform', 3026),
 ('Drink', 2903),
 ('Doll', 2712),
 ('Truck', 2620),
 ('Skyscraper', 2599),
 ('Cat', 2586),
 ('Insect', 2527),
 ('Baked goods', 2498),
 ('Train', 2448),
 ('Swimming pool', 2448),
 ('Human hair', 2428),
 ('Jeans', 2394),
 ('Butterfly', 2376),
 ('Picture frame', 2333),
 ('Land vehicle', 2281),
 ('Jacket', 2268),
 ('Door', 2132),
 ('Couch', 2009),
 ('Billboard', 1944),
 ('Houseplan

1736

In [120]:
import shutil

bird_ids = label_name_to_image_ids['Bird']
print(len(bird_ids))

osp.join(DATA, f'{split}/{image_id}.jpg')
os.makedirs(osp.join(DATA, f'easy_images/bird'), exist_ok=True)

for image_id in bird_ids:
    src = osp.join(DATA, f'{split}/{image_id}.jpg')
    dest = osp.join(DATA, f'easy_images/bird/{image_id}.jpg')
    shutil.copyfile(src, dest)
    
# Bicycle
bicycle_ids = label_name_to_image_ids['Bicycle']
print(len(bicycle_ids))

osp.join(DATA, f'{split}/{image_id}.jpg')
os.makedirs(osp.join(DATA, f'easy_images/bicycle'), exist_ok=True)

for image_id in bicycle_ids:
    src = osp.join(DATA, f'{split}/{image_id}.jpg')
    dest = osp.join(DATA, f'easy_images/bicycle/{image_id}.jpg')
    shutil.copyfile(src, dest)

3412
1321


In [121]:
!find {DATA}/easy_images/bird -name "*.jpg" | parallel --no-notice --bar -P 16 'convert {} -resize "299^>" -gravity center -crop 299x299+0+0 {}'

0% 0:3412=0s /root/cvd_open_images/easy_images/bird/ffda8fa36fc7cdf8.jpg                                                                                                               

100% 3412:0=0s /root/cvd_open_images/easy_images/bird/eac056e792f6a40d.jpg                                                                                                             


In [122]:
!find {DATA}/easy_images/bicycle -name "*.jpg" | parallel --no-notice --bar -P 16 'convert {} -resize "299^>" -gravity center -crop 299x299+0+0 {}'

0% 0:1321=0s /root/cvd_open_images/easy_images/bicycle/21429d11e39d7215.jpg                                                                                                            

100% 1321:0=0s /root/cvd_open_images/easy_images/bicycle/ce4e8f4fe8a5ae00.jpg                                                                                                          


In [125]:
!gsutil -m rsync -r {DATA}/easy_images gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped

Building synchronization state...
Starting synchronization
Copying file:///root/cvd_open_images/easy_images/bicycle/000f70501d048ef7.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/002f2d008de2f535.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/003d6a9ba866c4eb.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/004afd0c3cb926ab.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/0055a449c414ef9e.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/006fcabebcf65380.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/0061abb86f6f9797.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/007a3aae79a10a6c.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/007dd9372c1d4088.jpg [Conten

Copying file:///root/cvd_open_images/easy_images/bicycle/06f1d8047fbef3ba.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/0fc4b9ac5dda9b86.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/10fea1dfe66ef419.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/11eece52511d80e2.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/101423c022f7e66e.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/113fe5bfa24e4478.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/1244a3a4e92ecd68.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/1244ecf69d191d6c.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/126e2da88af58067.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/

Copying file:///root/cvd_open_images/easy_images/bicycle/225352fae2e4e861.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/22e6faf098267648.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/22ea9fd75df4873f.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/233e58b760fe1160.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/2388df2709dd1925.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/23ad84d4774ca965.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/23fc801fbe1e1716.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/24ac4da4d06bb10c.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/267e1edbdc4fd264.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/

Copying file:///root/cvd_open_images/easy_images/bicycle/371be2fa7e3a67cb.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/363478832c8c7cdc.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/3745f6e23efc1ad4.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/3704818d09a6687e.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/37e6a15259882305.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/3796ea787025122a.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/3811c481b76e4cf5.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/3781379c0643c9d5.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/3870102848c85c1d.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/

Copying file:///root/cvd_open_images/easy_images/bicycle/487bef2fe82dde99.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/493a5a8d0268f67d.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/48c6ed4dc9a25173.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/493ee5a73b5863de.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/4944837cf4d4548c.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/496e41bddce9df55.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/49ae2abf1f1e2037.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/4b53f4ed72e6abca.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bicycle/49ba29c5e81fa147.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/

...

Copying file:///root/cvd_open_images/easy_images/bird/de4789869c4e23e2.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/de8e23b2c03567be.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/dec04e9f3b16904c.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/de740622988bab9a.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/de9d50f2114f769b.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/dee3652a7ab0097e.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/def01082bf831aa2.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/defb4c991b77aa01.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/dee4a84be933250a.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/df1a2d5a97

Copying file:///root/cvd_open_images/easy_images/bird/e53e453aed651841.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/e54359ece3d6469c.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/e5c2ac6b038c7f55.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/e55b25bbafb5e316.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/e5d18b6ad55f1ac9.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/e609c8d27883b51d.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/e60f913d8d4e2815.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/e56ab75630c8ddba.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/e586d0971a422e7b.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/e6106ccfed

Copying file:///root/cvd_open_images/easy_images/bird/eddfb706f0301cde.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/ecfff8b6cd24a4d1.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/edf06a9927669741.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/edae9b5b4c9627e1.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/ed561dcdc3cb6e85.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/ee1358a227d7a60d.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/ee012abf6fe5a1a0.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/ee61a3b41735e3e5.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/ee2b89cf33a7bbff.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/ee227e8304

Copying file:///root/cvd_open_images/easy_images/bird/f5977685434d24b1.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/f5952b906498c475.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/f5b45dcdd77a9c05.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/f616e4fc0d3836b3.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/f5db1c2af71f5a5d.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/f5d147a5d826a120.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/f5f4916bd831ebd1.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/f626f3fb59ce5e6e.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/f5e45ff20f879a05.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/f6321ebb9a

Copying file:///root/cvd_open_images/easy_images/bird/fb30ea321cee3176.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/fb56cc0e35ca0b62.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/fba9400be2c76907.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/fbd15e42a1e7f805.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/fbbf0405de6fdf67.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/fbe17b71c92b046e.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/fbe3e840b364e4ad.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/fc2d743d402c27b1.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/fc17d1868af9a24c.jpg [Content-Type=image/jpeg]...
Copying file:///root/cvd_open_images/easy_images/bird/fbe9fc1f2d

In [126]:
!gsutil -m acl ch -u AllUsers:R  -r gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped

Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/000f70501d048ef7.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/007a3aae79a10a6c.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/002f2d008de2f535.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/006fcabebcf65380.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/004afd0c3cb926ab.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/0055a449c414ef9e.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/007ddcced8b5be79.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicyc

Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/0dc3b03608ea0fb0.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/06497030a88a9db7.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/108025dc07c2f8fe.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/1702f0215f63fb85.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/0530b7370532488c.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/1a2c405e5b32f99e.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/175d5548547ecd5b.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicyc

Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/0f810f9c8c74a880.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/1244ecf69d191d6c.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/1c6af59572d4baef.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/0c64982c033101f1.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/15a8ca30160cda04.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/1244a3a4e92ecd68.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/1d323a81de12dc98.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicyc

Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/21c0a9583a899eb8.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/23ad84d4774ca965.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/218e6ff9707cc3d2.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/0f1b9687e4bee13c.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/1d1d3c5f2e899630.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/28f44b5a5046aeb6.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/1569e9d0407e9d0c.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicyc

Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/3c177f4df3a0eb12.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/2ec45941c593eed1.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/3c4e795444495673.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/3e14fef0e04377e0.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/3c5af797f779ad0c.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/3e80c90e3b40cbd8.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/3ec0e2caec0ffeeb.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicyc

...

Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/de3f8019e3c381d2.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/e036c3f9519599f8.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/dfc8eb247cc6873d.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/df44f4806cefdd7e.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/e04047369c41ad70.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/df371b1e08de668b.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/dfed443255021f4e.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/e06a9166ab7a7e6a.jpg


Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/e75eb41d6bc630cb.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/e539c0dc94f7dbb6.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/e7acc83ba5d4cd04.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/e8f362eab40501b9.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/e86775b1a1a63bfc.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/e7f06437a579e120.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/e9310a5dce5bbb0f.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/e8cb9135a37e774d.jpg


Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/ef7f779a1dedff71.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/eed3e9bda364189c.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/ef1085830abe5190.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/ef987e42161fa22c.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/ef442746f4f05ea9.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/ef1ff40c6e7b3451.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/ef11b88b1d63cdf8.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/ef14662cd1c2d115.jpg


Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/f6a1eea718b71f4f.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/f707cf2939601fa7.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/f70ac85f19758d01.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/f738029d336d48fa.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/f6b97c78adb77eba.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/f68195c89ca38e77.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/f6e970d1a1e05045.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/f6e3bc0493c11e8e.jpg


Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/fdddd3d01fc0e730.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/fe353c9d963cafa1.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/fe58687bfdbfe537.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/fd48d025e6eb35e0.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/fe5c81d0c78d8785.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/ff18df3368add74f.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/fe7f2fb794d34297.jpg
Updated ACL on gs://tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/fe31521d51a76e98.jpg


# Make crowd compute dataset

In [137]:
from IPython.core.display import display, HTML
html = lambda s: display(HTML(s))
def to_url(dirname, filename):
    return f"https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/{dirname}/{filename}.jpg"
    
def show_by_url(url):
    html(f"<img src='{url}'></img>")
    
print(to_url('bird', list(bird_ids)[1]))
show_by_url(to_url('bird', list(bird_ids)[1]))
print(to_url('bicycle', list(bicycle_ids)[1]))
show_by_url(to_url('bicycle', list(bicycle_ids)[1]))

https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/9235511b994b6f10.jpg


https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/78085e19e3935fe5.jpg


In [143]:
all_images = []
for image_id in list(bird_ids)[:2000]:
    all_images.append(to_url('bird', image_id))
for image_id in list(bicycle_ids)[:2000]:
    all_images.append(to_url('bicycle', image_id))


In [144]:
import random
random.shuffle(all_images)
all_images[:10]

['https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/2d19f3e3bf208e0c.jpg',
 'https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/e634843a72e6bcf2.jpg',
 'https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/51cc22626995c769.jpg',
 'https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/8b1d8cff6f79017b.jpg',
 'https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/f349bda40b1ccc40.jpg',
 'https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/2ba1db70a9a024c6.jpg',
 'https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/0cb92be1c3823ba6.jpg',


In [145]:
with open('easy_images_bird_or_bicycle_cropped.csv','w') as f:
    for url in all_images:
        f.write(url+'\n')

In [1]:
!head easy_images_bird_or_bicycle_cropped.csv

https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/2d19f3e3bf208e0c.jpg
https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/e634843a72e6bcf2.jpg
https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/51cc22626995c769.jpg
https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/8b1d8cff6f79017b.jpg
https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/f349bda40b1ccc40.jpg
https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bicycle/2ba1db70a9a024c6.jpg
https://storage.cloud.google.com/tomfeelslucky-experiments/datasets/unambiguous-open-images/easy_images_cropped/bird/0cb92be1c3823ba6.jpg
https://storage.cloud.google

In [147]:
!wc -l easy_images_bird_or_bicycle_cropped.csv

3321 easy_images_bird_or_bicycle_cropped.csv
